<a href="https://colab.research.google.com/github/Ashleyyyyy567/Breast-Cancer-Classification/blob/master/pytorch_cnn_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Imports

In [ ]:
import numpy as np

from tqdm.notebook import tqdm as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, imshow, axis

#### Experimental settings

In [ ]:
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 20 # number of times which the entire dataset is passed throughout the model
batch_size = 64 # the size of input data took for one iteration
lr = 1e-3 # size of step

#### Download training data

In [ ]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

#### DataLoader example

In [ ]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

#### Sample and view dataset 

In [ ]:
fig = figure(figsize=(30, 15))
for i in range(1,10):
    a=fig.add_subplot(1,10,i)
    image, label = train_gen.dataset[i]
    imshow(np.squeeze(image), cmap='gray')
    a.set_title('label: {}'.format(label))
    axis('off')

#### Define neural network module using PyTorch nn api

$$
n_{\textrm{out}} = \textrm{Floor}\left[\frac{n_{\textrm{in}}+2\textrm{pad}-\textrm{kernel size}}{\textrm{stride}}\right]
$$
where $n$ is the number of input features

In [ ]:
class Net(nn.Module):
  def __init__(self, num_classes):
    super(Net,self).__init__()

    self.conv_layers = nn.Sequential(
        nn.Conv2d(1, 10, kernel_size=5, stride=1, padding=0),
        nn.MaxPool2d(2),
        nn.ReLU(),
    )
    self.fc_layers = nn.Sequential(
        nn.Linear(int(10*((28-(5-1))/2)**2), num_classes),
    )
  
  def forward(self,x):
    out = self.conv_layers(x)
    activations = out
    out = out.view(-1, int(10*((28-(5-1))/2)**2))
    out = self.fc_layers(out)
    return out, activations

#### Instantiate network

In [ ]:
net = Net(num_classes)
if torch.cuda.is_available():
  net.cuda()

#### Instantiate loss and optimizer objects

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

#### Training loop

In [ ]:
tr_loss_hist = []
val_loss_hist= []
lamb = 1e-5
for epoch in tqdm(range(num_epochs)):
  for i ,(images,labels) in enumerate(train_gen):
    images = images.cuda()
    labels = labels.cuda()
    
    optimizer.zero_grad()
    outputs,_ = net(images)
    loss = loss_function(outputs, labels)

    # L2 regularization
    if True:
      l2_reg = torch.tensor(0.).cuda()
      for param in net.parameters():
          if len(param.shape) > 1:
            l2_reg += torch.norm(param,p='fro')
          else:
            l2_reg += torch.norm(param,p=2)
      loss += lamb * l2_reg

    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      tqdm.write('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))
  with torch.no_grad():
    tr_images, tr_labels = next(iter(train_gen))
    val_images, val_labels = next(iter(test_gen))
    tr_output,_ = net(tr_images.cuda())
    val_output,_ = net(val_images.cuda())
    tr_loss = loss_function(tr_output, tr_labels.cuda())
    val_loss = loss_function(val_output, val_labels.cuda())
    tr_loss_hist.append(tr_loss.item())
    val_loss_hist.append(val_loss.item())

#### Model persistence

In [ ]:
torch.save(net.state_dict(), './model.pt')

net = Net(num_classes)
net.load_state_dict(torch.load('./model.pt'))
net.cuda()

In [ ]:
state_dict = torch.load('./model.pt')

#### Plot training and validation loss

In [ ]:
plt.plot(tr_loss_hist, label='training loss')
plt.plot(val_loss_hist, label='validation loss')
plt.legend()
plt.show()

#### Testing loop

In [ ]:
correct = 0
total = 0
incorrect_images = []
for images,labels in test_gen:
  images = images.cuda()
  labels = labels.cuda()
  
  output,_ = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  if any (predicted!= labels):
    incorrect_images.append(images.data[predicted != labels])
  total += labels.size(0)

incorrect_images = torch.vstack(incorrect_images).reshape((-1,28,28))
print('Accuracy of the model: %.3f' %((correct)/(total+1)))

In [ ]:
incorrect_images.shape

#### Visualize misclassified samples

In [ ]:
fig = figure(figsize=(30, 15))
for i in range(1,10):
    a=fig.add_subplot(1,10,i)
    image = incorrect_images[i]
    output,activations = net(torch.unsqueeze(torch.unsqueeze(image, 0),0))
    _, label = torch.max(output,1)

    imshow(np.squeeze(image.cpu().reshape((28,28))), cmap='gray')
    a.set_title('prediction: {}'.format(label.item()))
    axis('off')

### Investigating filter weights and activations

In [ ]:
first_layer_weight = net.conv_layers[0].weight.cpu().detach().numpy()
first_layer_weight.shape

In [ ]:
fig = figure(figsize=(30, 15))
plt.tight_layout()
for i in range(1,10):
    a=fig.add_subplot(2,10,i)
    weight = first_layer_weight[i,0,:]
    imshow(weight, cmap='gray')
    axis('off')
    
fig = figure(figsize=(30, 15))
plt.tight_layout()
for i in range(1,10):
    a=fig.add_subplot(2,10,i)
    imshow(activations[0][i].detach().cpu().numpy(), cmap='gray')
    axis('off')

fig = figure(figsize=(30, 15))
plt.tight_layout()
for i in range(1,10):
    a=fig.add_subplot(2,10,i)
    amax = np.argmax(activations[0][i].detach().cpu().numpy())
    amax = np.unravel_index(amax, activations[0][i].shape)
    actimg = image.cpu().numpy()
    minx,maxx = np.maximum(amax[0]*2-5,0),np.minimum(amax[0]*2+5,28)
    miny,maxy = np.maximum(amax[1]*2-5,0),np.minimum(amax[1]*2+5,28)
    actimg[minx:maxx, miny:maxy] += 1
    imshow(actimg, cmap='gray')
    axis('off')

In [ ]:
activation_list = []
for images,labels in test_gen:
  images = images.cuda()
  labels = labels.cuda()
  _,activations = net(images)
  activation_list.append(activations)

testset_activations = torch.vstack(activation_list)

In [ ]:
max_act = torch.argmax(torch.norm(testset_activations, dim=(-2,-1), p='fro'),dim=0)
max_act_images = test_gen.dataset.data[max_act]

In [ ]:
fig = figure(figsize=(30, 15))
plt.tight_layout()
for i in range(1,10):
    a=fig.add_subplot(2,10,i)
    imshow(max_act_images[i].detach().cpu().numpy(), cmap='gray')
    axis('off')